In [2]:
import torch
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
import tqdm
import copy
from matplotlib import pyplot as plt

In [3]:
init_temps = [270.15, 283.15, 296.15, 307.65, 318.65]
t_amb = [270.15, 283.15, 296.15, 307.65, 318.15]

In [6]:
ground_list = []
sim_list = []

for i in range(5):
    file_name = str(10*i) + '.csv'
    ground_list.append(pd.read_csv("nn_data/ground_{}".format(file_name)))
    temp_1 = ground_list[i]['temperature'].tolist()
    temp_1.insert(0, init_temps[i])
    ground_list[i]['t_amb'] = [t_amb[i]] * len(ground_list[i])
    ground_list[i]['lag_1'] = temp_1[:-1]
    ground_list[i]['lag_2'] = ground_list[i]['lag_1'].shift(1)
    ground_list[i]['lag_3'] = ground_list[i]['lag_2'].shift(1)
    ground_list[i]['lag_4'] = ground_list[i]['lag_3'].shift(1)
    ground_list[i]['lag_5'] = ground_list[i]['lag_4'].shift(1)
    ground_list[i]['rolling_mean'] = ground_list[i][[col for col in ground_list[i].columns if 'lag' in col]].mean(axis=1)
    ground_list[i]['t_amb'] = [295.15] * len(ground_list[i])
    
    sim_list.append(pd.read_csv("nn_data/dataset_{}".format(file_name))[1:])
    ground_list[i]['heat'] = sim_list[i]['heat'].tolist()

ground = pd.concat(ground_list, ignore_index=True)
sim = pd.concat(sim_list, ignore_index=True)

In [10]:
ground = ground.dropna()
ground

,current,voltage,temperature,power,time,t_amb,lag_1,lag_2,lag_3,lag_4,lag_5,rolling_mean,heat
4,1.0421,4.1499,270.1550,4.324527,4.0,295.15,270.1628,270.1142,270.1470,270.1026,270.1500,270.13532,0.006628
5,1.0035,4.1499,270.0871,4.164567,5.0,295.15,270.1550,270.1628,270.1142,270.1470,270.1026,270.13632,0.006163
6,0.9859,4.1500,270.1139,4.091665,6.0,295.15,270.0871,270.1550,270.1628,270.1142,270.1470,270.13322,0.005966
7,0.9726,4.1500,270.1571,4.036543,7.0,295.15,270.1139,270.0871,270.1550,270.1628,270.1142,270.12660,0.005825
8,0.9717,4.1500,270.1591,4.032707,8.0,295.15,270.1571,270.1139,270.0871,270.1550,270.1628,270.13518,0.005833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163606,0.2955,4.1500,319.0964,1.226373,34457.0,295.15,319.0743,319.0925,319.1025,319.0847,319.0953,319.08986,0.000291
163607,0.2741,4.1499,319.1230,1.137486,34458.0,295.15,319.0964,319.0743,319.0925,319.1025,319.0847,319.09008,0.000269
163608,0.2881,4.1500,319.1032,1.195515,34459.0,295.15,319.1230,319.0964,319.0743,319.0925,319.1025,319.09774,0.000283
163609,0.2687,4.1500,319.1166,1.115051,34460.0,295.15,319.1032,319.1230,319.0964,319.0743,319.0925,319.09788,0.000263


In [11]:
df_train = ground.copy()

df_train = df_train.drop(['power', 'time', 'voltage'], axis=1)
df_train.to_csv("train.csv", index=False)

df_train

,current,temperature,t_amb,lag_1,lag_2,lag_3,lag_4,lag_5,rolling_mean,heat
4,1.0421,270.1550,295.15,270.1628,270.1142,270.1470,270.1026,270.1500,270.13532,0.006628
5,1.0035,270.0871,295.15,270.1550,270.1628,270.1142,270.1470,270.1026,270.13632,0.006163
6,0.9859,270.1139,295.15,270.0871,270.1550,270.1628,270.1142,270.1470,270.13322,0.005966
7,0.9726,270.1571,295.15,270.1139,270.0871,270.1550,270.1628,270.1142,270.12660,0.005825
8,0.9717,270.1591,295.15,270.1571,270.1139,270.0871,270.1550,270.1628,270.13518,0.005833
...,...,...,...,...,...,...,...,...,...,...
163606,0.2955,319.0964,295.15,319.0743,319.0925,319.1025,319.0847,319.0953,319.08986,0.000291
163607,0.2741,319.1230,295.15,319.0964,319.0743,319.0925,319.1025,319.0847,319.09008,0.000269
163608,0.2881,319.1032,295.15,319.1230,319.0964,319.0743,319.0925,319.1025,319.09774,0.000283
163609,0.2687,319.1166,295.15,319.1032,319.1230,319.0964,319.0743,319.0925,319.09788,0.000263


### Define the model

In [11]:
import torch.nn as nn
import torch.nn as nn
import torch.optim as optim

 
# Define the model
model = nn.Sequential(
    nn.Linear(5, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
)
model.to('mps')

Sequential(
  (0): Linear(in_features=3, out_features=9, bias=True)
  (1): ReLU()
  (2): Linear(in_features=9, out_features=1, bias=True)
)

In [12]:
# loss function and optimizer
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

**TRAIN and TEST sets**

In [35]:
df_train = ground.copy()

df_train = df_train.drop('power', axis=1)
df_train = df_train.drop('time', axis=1)
df_train.to_csv("train.csv", index=False)

df_train

,current,voltage,temperature,temp-1,t_amb,heat
0,1.6718,4.1507,270.1026,270.1500,270.15,0.016959
1,1.0860,4.1500,270.1470,270.1026,270.15,0.007164
2,1.0452,4.1499,270.1142,270.1470,270.15,0.006644
3,1.0497,4.1500,270.1628,270.1142,270.15,0.006712
4,1.0421,4.1499,270.1550,270.1628,270.15,0.006628
...,...,...,...,...,...,...
163606,0.2955,4.1500,319.0964,319.0743,318.15,0.000291
163607,0.2741,4.1499,319.1230,319.0964,318.15,0.000269
163608,0.2881,4.1500,319.1032,319.1230,318.15,0.000283
163609,0.2687,4.1500,319.1166,319.1032,318.15,0.000263


In [40]:
df_test = df_train['temperature']

In [41]:
df_train = df_train.drop('temperature', axis=1)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df_train, df_test, train_size=0.8, shuffle=True)
X_train = torch.tensor(X_train.values, dtype=torch.float32).to('mps')
y_train = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1).to('mps')
X_test = torch.tensor(X_test.values, dtype=torch.float32).to('mps')
y_test = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1).to('mps')

In [43]:
# training parameters
n_epochs = 50   # number of epochs to run
batch_size = 10  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)

# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []

In [44]:
# training loop
for epoch in range(n_epochs):
    model.train()
    
    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(mse=float(loss))
    
    # evaluate accuracy at end of each epoch
    model.eval()
    y_pred = model(X_test)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    history.append(mse)
    
    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(model.state_dict())

# restore model and return best accuracy
model.load_state_dict(best_weights)
torch.save(model.state_dict(), 'nn_model_weights.pth')

Epoch 17:  23%|████████████████████████████████████████████████████████▉                                                                                                                                                                                           | 3052/13089 [00:10<00:34, 288.30batch/s, mse=581]

In [ ]:
print("MSE: %.2f" % best_mse)
print("RMSE: %.2f" % np.sqrt(best_mse))
plt.plot(history)
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    # Test out inference with 5 samples
    for i in range(5):
        X_sample = X_test[i: i+1]
        X_sample = torch.tensor(X_sample, dtype=torch.float32)
        y_pred = model(X_sample)
        print(f"{X_test[i].cpu()} -> {y_pred[0].cpu().numpy()} (expected {y_test[i].cpu().numpy()})")